In [1]:
import pandas as pd
import json

rival_pop_size = 8
num_battles = 10
total_battles = (rival_pop_size - 1) * num_battles

In [2]:
rival_results = json.load(open("..\\GpResults\\bestRivals.json"))
can_move_results = pd.DataFrame(rival_results['canMove'])
can_move_results['Type'] = 'CanMove'
cannot_move_results = pd.DataFrame(rival_results['cannotMove'])
cannot_move_results['Type'] = 'CannotMove'
merged_non_expanded = pd.concat([can_move_results, cannot_move_results])


In [3]:
expanded_can_move = pd.DataFrame(can_move_results)['fitnessLexicographic'].apply(pd.Series)
expanded_can_move['Type'] = 'CanMove'
expanded_can_move

expanded_cannot_move = pd.DataFrame(cannot_move_results)['fitnessLexicographic'].apply(pd.Series)
expanded_cannot_move['Type'] = 'CannotMove'
expanded_cannot_move

merged = pd.concat([expanded_cannot_move, expanded_can_move])
merged.columns = ['Victories', "NumberAlwaysWin", "TurnsToWin", "NumberAlwaysLose", "TurnsToLose", "Type"]
merged

,Victories,NumberAlwaysWin,TurnsToWin,NumberAlwaysLose,TurnsToLose,Type
0,37.0,7.0,132.026667,0.0,173.986667,CannotMove
1,46.0,7.0,182.740000,0.0,127.200000,CannotMove
2,38.0,7.0,149.133333,1.0,157.126667,CannotMove
3,55.0,7.0,207.760000,0.0,93.140000,CannotMove
4,50.0,7.0,199.860000,0.0,110.573333,CannotMove
5,51.0,7.0,197.853333,0.0,111.493333,CannotMove
6,51.0,7.0,206.660000,0.0,116.440000,CannotMove
7,39.0,7.0,159.200000,0.0,139.246667,CannotMove
0,39.0,7.0,143.293333,0.0,158.173333,CanMove
1,34.0,7.0,130.840000,0.0,171.880000,CanMove


In [4]:
merged_dropped = merged.drop(['NumberAlwaysWin', 'NumberAlwaysLose'], axis=1)

In [5]:
merged_dropped['TurnsToWin'] *= total_battles
merged_dropped['TurnsToLose'] *= total_battles
merged_dropped['TurnsToWin'] /= merged_dropped.Victories
merged_dropped['TurnsToLose'] /= merged_dropped.Victories.apply(lambda x: total_battles - x)

In [6]:
merged_dropped

,Victories,TurnsToWin,TurnsToLose,Type
0,37.0,249.780180,369.062626,CannotMove
1,46.0,278.082609,371.000000,CannotMove
2,38.0,274.719298,343.714583,CannotMove
3,55.0,264.421818,434.653333,CannotMove
4,50.0,279.804000,387.006667,CannotMove
5,51.0,271.563399,410.764912,CannotMove
6,51.0,283.650980,428.989474,CannotMove
7,39.0,285.743590,314.427957,CannotMove
0,39.0,257.193162,357.165591,CanMove
1,34.0,269.376471,334.211111,CanMove


In [8]:
from scipy.stats import kruskal

can_move = merged_dropped[merged_dropped['Type'] == 'CanMove']
cannot_move = merged_dropped[merged_dropped['Type'] == 'CannotMove']

print(kruskal(can_move.Victories, cannot_move.Victories))

KruskalResult(statistic=5.369822485207106, pvalue=0.020488062527199803)
KruskalResult(statistic=2.161764705882355, pvalue=0.141482121482789)
KruskalResult(statistic=3.57352941176471, pvalue=0.05870740843120496)


In [332]:
import functools

min_max_mapping = [1, 0, 0, 1, 1]
def compare(v1, v2):
    for d in range(len(v1)):
        if v1[d] == v2[d]:
            continue

        if min_max_mapping[d] == 1:
            if v1[d] > v2[d]:
                return -1
            else:
                return 1
        else:
            if v1[d] < v2[d]:
                return -1
            else:
                return 1
    return 0


scores = merged_non_expanded.fitnessLexicographic.to_list()

scores.sort(key=functools.cmp_to_key(compare))
map_type = []
for score in scores:
    for _, row in merged_non_expanded.iterrows():
        if row['fitnessLexicographic'] == score:
            map_type.append(row['Type'])

can_moves = [i for i in range(len(map_type)) if map_type[i] == 'CanMove']

cannot_moves = [i for i in range(len(map_type)) if map_type[i] == 'CannotMove']

print("CanMove")
display(pd.Series(can_moves).describe())
print("Cannotmove")
display(pd.Series(cannot_moves).describe())

CanMove


count     8.00000
mean     10.12500
std       4.94072
min       0.00000
25%       9.00000
50%      11.50000
75%      13.25000
max      15.00000
dtype: float64

Cannotmove


count    8.000000
mean     4.875000
std      2.900123
min      1.000000
25%      2.750000
50%      4.500000
75%      7.250000
max      9.000000
dtype: float64